In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evoman.environment import Environment
from controller_custom import player_controller
from multiprocessing import Pool
from scipy.stats import ttest_ind, mannwhitneyu
import pandas as pd
import warnings

pygame 2.6.0 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Algorithm A

In [3]:
def run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group, elitism_count=1):
	"""
	Runs Genetic Algorithm A with the provided environment and parameters.

	Parameters:
	- env: Evoman Environment instance.
	- npopulation: Population size.
	- gens: Number of generations.
	- mutation_rate: Mutation rate.
	- dom_u: Upper bound for gene values.
	- dom_l: Lower bound for gene values.
	- elitism_count: Number of elites to preserve.

	Returns:
	- history_mean: List of mean fitness per generation.
	- history_max: List of max fitness per generation.
	- best_solution: Best solution found.
	"""
	
	n_hidden_neurons = 10
	n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
	
	# Run the simulation and return the fitness
	def simulate(x):
		env.player_controller.set(x, env.get_num_sensors())
		f, _, _, _ = env.play(pcont=x)
		return f
	
	# Evaluate the current population
	def evaluate(population):
		return np.array([simulate(individual) for individual in population])

	# Tournament Selection
	def tournament_selection(population, fitness, k=5):
		selected = []
		for _ in range(len(population)):
			contenders = np.random.choice(len(population), k, replace=False)
			winner = contenders[np.argmax(fitness[contenders])]
			selected.append(population[winner])
		return np.array(selected)

	# Uniform Crossover
	def crossover(parent1, parent2):
		mask = np.random.rand(n_vars) < 0.5
		child1 = np.where(mask, parent1, parent2)
		child2 = np.where(mask, parent2, parent1)
		return child1, child2

	# Gaussian Mutation
	def mutate(child):
		for i in range(n_vars):
			if np.random.rand() < mutation_rate:
				child[i] += np.random.normal(0, 0.1)
				child[i] = np.clip(child[i], dom_l, dom_u)
		return child

	# Initialize population
	population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
	fitness = evaluate(population)

	# Record fitness over generations
	history_mean = []
	history_max = []

	# Genetic Algorithm Loop
	for generation in range(1, gens + 1):
		# Selection
		selected = tournament_selection(population, fitness)

		# Crossover
		offspring = []
		for i in range(0, npopulation, 2):
			parent1, parent2 = selected[i], selected[i+1]
			child1, child2 = crossover(parent1, parent2)
			offspring.extend([child1, child2])
		offspring = np.array(offspring)[:npopulation]

		# Mutation
		offspring = np.array([mutate(child) for child in offspring])

		# Evaluation
		offspring_fitness = evaluate(offspring)

		# Replacement: Elitism (preserve top elitism_count individuals)
		elite_indices = np.argsort(fitness)[-elitism_count:]  
		elite_individuals = population[elite_indices]  
		elite_fitness = fitness[elite_indices]  
		worst_indices = np.argsort(offspring_fitness)[:elitism_count]  
		offspring[worst_indices] = elite_individuals  
		offspring_fitness[worst_indices] = elite_fitness  # Replace the fitness of the worst with elite fitness

		# Update population and fitness
		population, fitness = offspring, offspring_fitness

		# Record statistics
		history_mean.append(np.mean(fitness))
		history_max.append(np.max(fitness))

		# Logging
		print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}')

	# Get best solution
	best_idx = np.argmax(fitness)
	best_solution = population[best_idx]

	np.savetxt(os.path.join('results', f'best_solution_A_{enemy_group}.txt'), best_solution)

	return history_mean, history_max, best_solution


# Algorithm B

In [4]:
def run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group):
    """
    Runs Genetic Algorithm B with Crowding for preserving diversity.

    Parameters:
    - env: Evoman Environment instance.
    - npopulation: Population size.
    - gens: Number of generations.
    - mutation_rate: Mutation rate.
    - dom_u: Upper bound for gene values.
    - dom_l: Lower bound for gene values.
    - crowding_factor: Factor to determine the number of individuals to compete.

    Returns:
    - history_mean: List of mean fitness per generation.
    - history_max: List of max fitness per generation.
    - best_solution: Best solution found.
    """
    
    n_hidden_neurons = 10
    n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5

    # Run the simulation and return the fitness
    def simulate(x):
        env.player_controller.set(x, env.get_num_sensors())
        f, _, _, _ = env.play(pcont=x)
        return f

    # Evaluate the current population
    def evaluate(population):
        return np.array([simulate(individual) for individual in population])

    # Tournament Selection
    def tournament_selection(population, fitness, k=5):
        selected = []
        for _ in range(len(population)):
            contenders = np.random.choice(len(population), k, replace=False)
            winner = contenders[np.argmax(fitness[contenders])]
            selected.append(population[winner])
        return np.array(selected)

    # Uniform Crossover
    def crossover(parent1, parent2):
        mask = np.random.rand(n_vars) < 0.5
        child1 = np.where(mask, parent1, parent2)
        child2 = np.where(mask, parent2, parent1)
        return child1, child2

    # Gaussian Mutation
    def mutate(child):
        for i in range(n_vars):
            if np.random.rand() < mutation_rate:
                child[i] += np.random.normal(0, 0.1)
                child[i] = np.clip(child[i], dom_l, dom_u)
        return child


    # Initialize population
    population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
    fitness = evaluate(population)

    # Record fitness over generations
    history_mean = []
    history_max = []
    history_std = []

    # Genetic Algorithm Loop
    for generation in range(1, gens + 1):
        # Selection
        selected = tournament_selection(population, fitness)

        # Crossover 
        for i in range(0, npopulation, 2):
            parent1, parent2 = selected[i], selected[i + 1]
            child1, child2 = crossover(parent1, parent2)
            
            # Mutation
            child1 = mutate(child1)
            child2 = mutate(child2)

            # Calculate distances (Euclidean) between parents and offspring
            d_p1_o1 = np.linalg.norm(parent1 - child1)
            d_p1_o2 = np.linalg.norm(parent1 - child2)
            d_p2_o1 = np.linalg.norm(parent2 - child1)
            d_p2_o2 = np.linalg.norm(parent2 - child2)

            # Calculate fitness for offspring
            fitness_child1 = simulate(child1)
            fitness_child2 = simulate(child2)

            # Crowding
            if d_p1_o1 + d_p2_o2 < d_p1_o2 + d_p2_o1:
                # Pair child1 with parent1 and child2 with parent2
                if fitness_child1 > fitness[i]:
                    population[i] = child1
                if fitness_child2 > fitness[i + 1]:
                    population[i + 1] = child2
            else:
                # Pair child1 with parent2 and child2 with parent1
                if fitness_child1 > fitness[i + 1]:
                    population[i + 1] = child1
                if fitness_child2 > fitness[i]:
                    population[i] = child2 

        # Recalculate fitness for the updated population
        fitness = evaluate(population)

        # Record statistics for the generation
        history_mean.append(np.mean(fitness))
        history_max.append(np.max(fitness))
        history_std.append(np.std(fitness))

        # Logging
        print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}, Std = {history_std[-1]:.4f}')

    # Get best solution
    best_idx = np.argmax(fitness)
    best_solution = population[best_idx]

    np.savetxt(os.path.join('results', f'best_solution_B_{enemy_group}.txt'), best_solution)

    return history_mean, history_max, best_solution


In [5]:
def run_algorithm_b_with_elitism(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group, elitism_count=2):
    """
    Runs Genetic Algorithm B with Crowding and Elitism for preserving diversity and best solutions.
    """
    
    n_hidden_neurons = 10
    n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5

    def simulate(x):
        env.player_controller.set(x, env.get_num_sensors())
        f, _, _, _ = env.play(pcont=x)
        return f

    def evaluate(population):
        return np.array([simulate(individual) for individual in population])

    def tournament_selection(population, fitness, k=5):
        selected = []
        for _ in range(len(population)):
            contenders = np.random.choice(len(population), k, replace=False)
            winner = contenders[np.argmax(fitness[contenders])]
            selected.append(population[winner])
        return np.array(selected)

    def crossover(parent1, parent2):
        mask = np.random.rand(n_vars) < 0.5
        child1 = np.where(mask, parent1, parent2)
        child2 = np.where(mask, parent2, parent1)
        return child1, child2

    def mutate(child):
        for i in range(n_vars):
            if np.random.rand() < mutation_rate:
                child[i] += np.random.normal(0, 0.1)
                child[i] = np.clip(child[i], dom_l, dom_u)
        return child

    def crowding_replace(population, selected, offspring, fitness):
        for i in range(len(selected) // 2):
            parent1, parent2 = selected[2 * i], selected[2 * i + 1]
            child1, child2 = offspring[2 * i], offspring[2 * i + 1]

            d_p1_o1 = np.linalg.norm(parent1 - child1)
            d_p2_o2 = np.linalg.norm(parent2 - child2)
            d_p1_o2 = np.linalg.norm(parent1 - child2)
            d_p2_o1 = np.linalg.norm(parent2 - child1)

            if d_p1_o1 + d_p2_o2 < d_p1_o2 + d_p2_o1:
                if simulate(child1) > simulate(parent1):
                    population[2 * i] = child1
                    fitness[2 * i] = simulate(child1)
                if simulate(child2) > simulate(parent2):
                    population[2 * i + 1] = child2
                    fitness[2 * i + 1] = simulate(child2)
            else:
                if simulate(child1) > simulate(parent2):
                    population[2 * i + 1] = child1
                    fitness[2 * i + 1] = simulate(child1)
                if simulate(child2) > simulate(parent1):
                    population[2 * i] = child2
                    fitness[2 * i] = simulate(child2)

        return population, fitness

    # Elitism Function
    def apply_elitism(population, fitness, num_elites=elitism_count):
        elite_indices = np.argsort(fitness)[-num_elites:]
        elites = population[elite_indices]
        elites_fitness = fitness[elite_indices]
        return elites, elites_fitness

    # Initialize population
    population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
    fitness = evaluate(population)

    history_mean = []
    history_max = []
    history_std = []

    for generation in range(1, gens + 1):
        selected = tournament_selection(population, fitness)

        offspring = []
        for i in range(0, npopulation, 2):
            parent1, parent2 = selected[i], selected[i + 1]
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])
        offspring = np.array(offspring)[:npopulation]

        offspring = np.array([mutate(child) for child in offspring])

        elites, elites_fitness = apply_elitism(population, fitness)
        population, fitness = crowding_replace(population, selected, offspring, fitness)
        population[:elitism_count] = elites
        fitness[:elitism_count] = elites_fitness

        history_mean.append(np.mean(fitness))
        history_max.append(np.max(fitness))
        history_std.append(np.std(fitness))

        print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}, Std = {history_std[-1]:.4f}')

    best_idx = np.argmax(fitness)
    best_solution = population[best_idx]

    np.savetxt(os.path.join('results', f'best_solution_B_{enemy_group}.txt'), best_solution)

    return history_mean, history_max, best_solution


# Parameters 

In [4]:
# Experiment Parameters

enemy_group_1 = [1, 4, 5, 8]
enemy_group_2 = [2, 3, 6, 7]

num_runs = 10
npopulation = 100
gens = 30 				
mutation_rate = 0.01
dom_u, dom_l = 1, -1
elitism_count_a = 1
elitism_count_b = 2
 

# The number of hidden neurons for the player controller
n_hidden_neurons = 10

# Initialize data storage
results = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Generation': [],
    'Mean Fitness': [],
    'Max Fitness': [],
    'Best Solution': []
}

best_solutions = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Best Solution': []
}

# Run Algorithm A on Enemy Group 1

In [5]:
print('Running Algorithm A for enemy group 1')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA1_EG1'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_1,
						playermode="ai",
						multiplemode="yes",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_1')

	for gen in range(gens):
		results['Algorithm'].append('A')
		results['Enemy Group'].append(1)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('A')
	best_solutions['Enemy Group'].append(1)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA1_EG1.csv'), index=False)

Running Algorithm A for enemy group 1
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 21.5080, Mean Fitness = -3.4399
Generation 2: Best Fitness = 30.3287, Mean Fitness = -0.4785
Generation 3: Best Fitness = 42.6920, Mean Fitness = 2.0795
Generation 4: Best Fitness = 42.6920, Mean Fitness = 3.4607
Generation 5: Best Fitness = 46.5809, Mean Fitness = 5.8257
Generation 6: Best Fitness = 46.5809, Mean Fitness = 8.1771
Generation 7: Best Fitness = 46.5809, Mean Fitness = 13.1395
Generation 8: Best Fitness = 49.8253, Mean Fitness = 13.6491
Generation 9: Best Fitness = 54.9897, Mean Fitness = 12.2105
Generation 10: Best Fitness = 54.9897, Mean Fitness = 15.8519
Generation 11: Best Fitness = 54.9897, Mean Fitness = 18.8825
Generation 12: Best Fitness = 66.8159, Mean Fitness = 20.2016
Generation 13: Best Fitness = 66.8159, Mean Fitness = 22.0139
Generation 14: Best Fitness = 66.8159, Mean Fitness = 27.0689
Generation 15: Best Fitness = 66.8159, Mean Fitness = 34

# Run Algorithm A on Enemy Group 2

In [ ]:
print('Running Algorithm A for enemy group 2')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA1_EG2'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_2,
						playermode="ai",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_2')

	for gen in range(gens):
		results['Algorithm'].append('A')
		results['Enemy Group'].append(2)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('A')
	best_solutions['Enemy Group'].append(2)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA1_EG2.csv'), index=False)

# Run Algorithm B on Enemy Group 1

In [8]:
print('Running Algorithm B for enemy group 1')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG1'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_1,
						playermode="ai",
						multiplemode="yes",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_1')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(1)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(1)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG1.csv'), index=False)

Running Algorithm B for enemy group 1
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 29.2752, Mean Fitness = -3.2095, Std = 6.3365
Generation 2: Best Fitness = 29.4226, Mean Fitness = 0.6081, Std = 8.5296
Generation 3: Best Fitness = 32.6376, Mean Fitness = 4.8913, Std = 9.5915
Generation 4: Best Fitness = 46.5586, Mean Fitness = 9.6498, Std = 9.7571
Generation 5: Best Fitness = 46.5586, Mean Fitness = 13.1632, Std = 10.2417
Generation 6: Best Fitness = 48.0151, Mean Fitness = 17.2190, Std = 10.4572
Generation 7: Best Fitness = 50.5218, Mean Fitness = 21.2574, Std = 11.0616
Generation 8: Best Fitness = 60.9435, Mean Fitness = 28.6136, Std = 11.2158
Generation 9: Best Fitness = 64.2278, Mean Fitness = 36.7448, Std = 9.3533
Generation 10: Best Fitness = 64.2278, Mean Fitness = 41.3074, Std = 8.7356
Run 2

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 25.8885, Mean Fitness = -3.2705, Std = 6.2370
Generation 2: Best Fitness = 49.8

# Run Algorithm B on Enemy Group 2

In [ ]:
print('Running Algorithm B for enemy group 2')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG2'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_2,
						playermode="ai",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_2')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(2)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(2)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG2.csv'), index=False)